In [2]:
from openai import OpenAI
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

client = OpenAI()


In [3]:
#1단계 : 가짜 날씨 함수
def get_weather(city, date):
  if city == "Seoul":
    return "-5"
  elif city == "Sidney":
    return "15"

In [4]:
#2단계 : tools 정의
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "도시 이름과 날짜를 받아 날씨를 조회한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "도시 이름 (예: Seoul, Tokyo)"
                    },
                    "date": {
                        "type": "string",
                        "description": "YYYY-MM-DD 형식의 날짜",
                    }
                },
                "required": ["city"],
                "additionalProperties": False,
            },
    }
]


In [5]:

user_question = "내일 서울 날씨 어때?"


# 3단계: tool_call 받기
resp = client.responses.create(
    model="gpt-4.1-mini",
    tools=tools,
    # instructions = "오늘 날짜는" + date +
    input=user_question,
)

print(resp.output)

[ResponseFunctionToolCall(arguments='{"city":"Seoul","date":"2024-04-28"}', call_id='call_fB1sGDHKUAw3tNKEpxw12s24', name='get_weather', type='function_call', id='fc_04314857e6daa4fe006954cb0ced24819fa193fb0b4c9b97e8', status='completed')]


In [6]:
import json

args = json.loads(resp.output[0].arguments)
print(args)

result = get_weather(**args)
print(result)

{'city': 'Seoul', 'date': '2024-04-28'}
-5


In [7]:
# 5단계: 첫 번째 호출: tool_call 받기
# 대화 히스토리를 리스트로 관리
input_items = [
    {
        "role": "user",
        "content": user_question,
    }
]

    # (1) 기존 대화에 tool_call 자체를 추가
input_items.append(resp.output[0])

# (2) function_call_output 아이템 추가
input_items.append(
    {
        "type": "function_call_output",
        "call_id": resp.output[0].call_id,
        "output": json.dumps(result, ensure_ascii=False),
    }
)

    # (3) 최종 답변 생성 요청
resp2 = client.responses.create(
    model="gpt-4.1-mini",
    tools=tools,
    input=input_items,
    instructions=(
        "너는 날씨 안내 봇이다. "
        "function_call_output 으로 전달된 날씨 정보를 활용해서, "
        "사용자에게 한국어로 자연스럽게 설명해줘."
    ),
)

print("\n=== 최종 답변 ===")
print(resp2.output_text)


=== 최종 답변 ===
내일 서울의 예상 최저 기온은 영하 5도입니다. 날씨가 꽤 춥겠으니, 외출하실 때 따뜻하게 입으시길 권장드립니다. 더 자세한 날씨 상태가 필요하시면 알려주세요!
